In [1]:
from pynwb import NWBFile
from neo import *
import pynwb
from pynwb import *
import quantities as pq
from quantities import s, ms,kHz, Hz, uV, nA
import numpy as np
from datetime import datetime
import nwbinspector

## Define Neo structure

In [2]:
annotations = {
            "session_start_time": datetime.now()
        }
# Define Neo blocks
bl0 = Block(name='First block', 
                    experimenter="Experimenter's name", 
                    experiment_description="Experiment description",
                    institution="Institution", 
                    **annotations)
bl1 = Block(name='Second block', 
                    experimenter="Experimenter's name", 
                    experiment_description="Experiment description",
                    institution="Institution", 
                    **annotations)
bl2 = Block(name='Third block', 
                    experimenter="Experimenter's name", 
                    experiment_description="Experiment description",
                    institution="Institution", 
                    **annotations)
#original_blocks = [bl0, bl1, bl2]
original_blocks = [bl0]

num_seg = 2 # number of segments
num_chan = 3 # number of channels

#for blk in original_blocks: # original
for j, blk in enumerate(original_blocks):

    for ind in range(num_seg):  # number of Segments
        seg = Segment(index=ind)
        seg.block = blk
        blk.segments.append(seg)

#    for seg in blk.segments:  # AnalogSignal objects # original
    for i, seg in enumerate(blk.segments):

        # 3 Neo AnalogSignals
        a = AnalogSignal(name='Signal_a %s' % (seg.name),
                                 signal=np.random.randn(44, num_chan) * pq.nA,
                                 sampling_rate=10 * pq.kHz,
                                 t_start=50 * pq.ms
                        )
        b = AnalogSignal(name='Signal_b %s' % (seg.name),
                                 signal=np.random.randn(64, num_chan) * pq.mV,
                                 sampling_rate=8 * pq.kHz,
                                 t_start=40 * pq.ms)
        c = AnalogSignal(name='Signal_c %s' % (seg.name),
                                signal=np.random.randn(33, num_chan) * pq.uA,
                                 sampling_rate=10 * pq.kHz,
                                t_start=120 * pq.ms)
        
        # 2 Neo IrregularlySampledSignals
        d = IrregularlySampledSignal([0.0, 1.23, 6.78], [1, 2, 3], units='mV', time_units='ms')    

        # 2 Neo SpikeTrains
        train = SpikeTrain(times=[1, 2, 3] * pq.s, t_start=1.0, t_stop=10.0)
        train2 = SpikeTrain(times=[4, 5, 6] * pq.s, t_stop=10.0)
 
        # 1 Neo Event
        #evt = Event(name='Event',
        #                   times=[0.01 + i*ind, 11 + i*ind, 33 + i*ind]*pq.s,
        #                   labels=np.array(['ev0', 'ev1', 'ev2']))
        
        evt = Event(name='Event', # original
                    times=np.arange(0, 30, 10) * pq.ms,
                    labels=np.array(['ev0', 'ev1', 'ev2']))
        

        # 2 Neo Epochs                
        epc = Epoch(times=[0 + i*ind, 10 + i*ind, 33 + i*ind]*pq.s,
                    durations=[10, 5, 7]*pq.s,
                    labels=np.array(['btn0', 'btn1', 'btn2'], dtype='U')
                   )

        """
        # Test for specific epoch:
        if i==0:
            epc = Epoch(times=[0 + i*ind, 10 + i*ind, 33 + i*ind]*pq.s,
                    durations=[10, 5, 7]*pq.s, # original
                    labels=np.array(['btn0', 'btn1', 'btn2'], dtype='U')
                   )
        if i==1:
            epc = Epoch(times=[0 + i*ind, 10 + i*ind, 33 + i*ind]*pq.s,
                    durations=[10, 5, 1]*pq.s, # original
                    labels=np.array(['btn0', 'btn1', 'btn2'], dtype='U')
                   )
        """ 

        epc2 = Epoch(times=[0.1 + i*ind, 30 + i*ind, 61 + i*ind]*pq.s,
                     durations=[10, 5, 7]*pq.s,
                     labels=np.array(['btn4', 'btn5', 'btn6']))


        seg.spiketrains.append(train)
        seg.spiketrains.append(train2)

        seg.epochs.append(epc)
        seg.epochs.append(epc2)

        seg.analogsignals.append(a)
        seg.analogsignals.append(b)
        seg.analogsignals.append(c)
        seg.irregularlysampledsignals.append(d)
        seg.events.append(evt)
        a.segment = seg
        b.segment = seg
        c.segment = seg
        d.segment = seg
        evt.segment = seg
        train.segment = seg
        train2.segment = seg
        epc.segment = seg
        epc2.segment = seg

## Write a NWB file using Neo NWBIO

In [3]:
test_file_name = "test_NWB_Inspector_File_without_Annotations.nwb"
iow = NWBIO(filename=test_file_name, mode='w')
iow.write_all_blocks(original_blocks)
print("iow = ", iow)

/Users/legouee/miniconda3/envs/Brainhack/lib/python3.8/site-packages/pynwb/file.py:1037: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")


iow =  <neo.io.nwbio.NWBIO object at 0x12c236a00>


## Read this file using Neo NWBIO

In [4]:
ior = NWBIO(filename=test_file_name, mode='r')
retrieved_block = ior.read_all_blocks()
print("retrieved_block = ", retrieved_block)

retrieved_block =  [<neo.core.block.Block object at 0x12c8cf4f0>]


## Validating NWB file using PyNWB

In [5]:
!python -m pynwb.validate test_NWB_Inspector_File_without_Annotations.nwb

Validating test_NWB_Inspector_File_without_Annotations.nwb against cached namespace information using namespace 'core'.
 - no errors found.


## NWB Inspector
### Inspect NWB files for compliance with NWB Best Practices. 

### Quick check

In [6]:
!nwbinspector /Users/legouee/NWBwork/my_notebook/test_NWB_Inspector_File_Without_Annotations.nwb

## Common flags

### Save the report for future reference

In [7]:
#!nwbinspector /Users/legouee/NWBwork/my_notebook/test_NWB_Inspector.nwb -o --log-file /Users/legouee/NWBwork/my_notebook/nwbinspector_report.txt
!nwbinspector /Users/legouee/NWBwork/my_notebook/test_NWB_Inspector_File_without_Annotations.nwb -o --json-file-path /Users/legouee/NWBwork/my_notebook/nwbinspector_report_File_without_Annotations.txt

## Inspect a single NWBFile

In [8]:
from nwbinspector import inspect_nwb

results = list(inspect_nwb(nwbfile_path="test_NWB_Inspector_File_without_Annotations.nwb"))
print("results = ", results)

results =  []


## Yielding and Iterating

In [9]:
results_generator = inspect_nwb(nwbfile_path="test_NWB_Inspector_File_without_Annotations.nwb")
print("results_generator = ", results_generator)

results_generator =  <generator object inspect_nwb at 0x12c96e5f0>


In [10]:
#first_message = next(results_generator)
#print("first_message = ", first_message)

In [11]:
results_generator = inspect_nwb(nwbfile_path="test_NWB_Inspector_File_without_Annotations.nwb")

for message in results_generator:
    print(message.message)
    print("  ")
    print(message)
    print("   ")
    print("message.importance=", message.importance)
    print("message.importance.__dict__=", message.importance.__dict__)
    
    print("message.severity=", message.severity)
    print("message.severity.__dict__=", message.severity.__dict__)
    print("message.severity=", message.severity)
    
    
    if message.importance._name_=="BEST_PRACTICE_SUGGESTION":
        print("ok")
    if message.severity._name_=="LOW":
        print("OKok")
    print("----")